In [87]:
import pandas as pd

In [2]:
import csv

In [3]:
import numpy as np
import numpy.ma as ma
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate

In [4]:
pd.set_option("display.precision", 1)

In [234]:
item_train = pd.read_csv('cleaned_converted_data.csv')
item_train

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [235]:
rating_df = pd.read_csv("rating.csv")
rating_df

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [236]:
user_train=pd.read_csv("user_genre_avg_ratings_subset_no_header.csv")
user_train = np.nan_to_num(user_train, nan=0.0)
user_train

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 7.29032258,  7.53191489,  0.        , ...,  6.        ,
         0.        ,  0.        ],
       [ 4.7012987 ,  5.18681319,  6.        , ...,  4.42857143,
         0.        ,  0.        ],
       ...,
       [ 7.92307692,  7.55555556,  0.        , ..., 10.        ,
         0.        ,  0.        ],
       [ 9.        ,  8.25      ,  0.        , ...,  7.        ,
         0.        ,  0.        ],
       [ 8.55555556,  8.66666667,  0.        , ...,  9.        ,
         0.        ,  0.        ]])

In [237]:
num_user_features = user_train.shape[1] 
num_item_features = item_train.shape[1]  
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 1  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")
print(f"Number of training vectors: {len(user_train)}")

Number of training vectors: 12294
Number of training vectors: 9466


In [238]:
y_train=pd.read_csv("rating_only.csv")
y_train

,9.37
0,9.3
1,9.2
2,9.2
3,9.2
4,9.2
...,...
12058,4.2
12059,4.3
12060,4.9
12061,5.0


In [239]:
item_train_unscaled = item_train
user_train_unscaled = user_train

y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)


scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(np.array(y_train).reshape(-1, 1))
y_train = scalerTarget.transform(np.array(y_train).reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))


print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [240]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")

print(f"movie/item test data shape: {item_test.shape}")
print(f"user training data shape: {user_train.shape}")
print(f"user test data shape: {user_test.shape}")

movie/item training data shape: (9835, 43)
movie/item test data shape: (2459, 43)
user training data shape: (7572, 43)
user test data shape: (1894, 43)


In [181]:
from tensorflow.keras.layers import Lambda

In [241]:
from tensorflow.keras.layers import Dense
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([     
         Dense(units=256,activation='relu'),
         Dense(units=128,activation='relu'),
         Dense(units=num_outputs,activation='linear')
    
])

item_NN = tf.keras.models.Sequential([    
        Dense(units=256,activation='relu'),
        Dense(units=128,activation='relu'),
        Dense(units=num_outputs,activation='linear')
  
  
])

input_user = tf.keras.layers.Input(shape=(num_user_features,))
vu = user_NN(input_user)
vu = Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vu)
input_item = tf.keras.layers.Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vm)

output = tf.keras.layers.Dot(axes=1)([vu, vm])


model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "functional_54"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_70 (InputLayer)   │ (None, 43)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_72 (InputLayer)   │ (None, 43)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential_36 (Sequential)    │ (None, 32)                │          48,288 │ input_layer_70[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential_37 (Sequential)    │ (None, 32)                │          48,288 │ input_layer_72[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lambda_34 (Lambda)            │ (None, 32)                │               0 │ sequential_36[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lambda_35 (Lambda)            │ (None, 32)                │               0 │ sequential_37[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dot_17 (Dot)                  │ (None, 1)                 │               0 │ lambda_34[0][0],           │
│                               │                           │                 │ lambda_35[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 96,576 (377.25 KB)

 Trainable params: 96,576 (377.25 KB)

 Non-trainable params: 0 (0.00 B)

In [242]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [243]:
print("user_train shape:", user_train.shape)
print("item_train shape:", item_train.shape)
print("y_train shape:", y_train.shape)



user_train shape: (7572, 43)
item_train shape: (9835, 43)
y_train shape: (9650, 1)


In [244]:
min_samples = min(user_train.shape[0], item_train.shape[0], y_train.shape[0])

user_train = user_train[:min_samples]
item_train = item_train[:min_samples]
y_train = y_train[:min_samples]

print(user_train.shape, item_train.shape, y_train.shape)  

(7572, 43) (7572, 43) (7572, 1)


In [186]:

X_user = user_train

X_item = item_train  #


In [187]:
print("User shape:", user_train.shape, type(user_train))
print("Item shape:", item_train.shape, type(item_train))
print("y_train shape:", y_train.shape)


User shape: (7573, 43) <class 'numpy.ndarray'>
Item shape: (7573, 43) <class 'numpy.ndarray'>
y_train shape: (7573, 1)


In [245]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',     
    patience=5,             
    restore_best_weights=True
)

model.fit(
    [user_train, item_train],
    y_train,
    validation_split=0.2,
    epochs=100,
    callbacks=[early_stop]
)

Epoch 1/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0805 - val_loss: 0.0607
Epoch 2/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0623 - val_loss: 0.0610
Epoch 3/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0617 - val_loss: 0.0611
Epoch 4/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0611 - val_loss: 0.0614
Epoch 5/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0607 - val_loss: 0.0613
Epoch 6/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0603 - val_loss: 0.0612


In [246]:
min_samples = min(user_test.shape[0], item_test.shape[0], y_test.shape[0])

user_test = user_test[:min_samples]
item_test = item_test[:min_samples]
y_test = y_test[:min_samples]

print(user_test.shape, item_test.shape, y_test.shape)

(1894, 43) (1894, 43) (1894, 1)


In [247]:
model.evaluate([user_test, item_test], y_test)

60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0652


0.06464970111846924

In [248]:
new_sh = pd.read_csv('converted_data.csv')
anime_df = pd.read_csv("anime_with_avg_rating.csv")
anime_dict = dict(zip(anime_df['anime_id'], anime_df['name']))
new_sh = anime_df.reset_index(drop=True)  

In [249]:
def split_str(ifeatures, smax):
    ''' split the feature name strings to tables fit '''
    ofeatures = []
    for s in ifeatures:
        if not ' ' in s:  
            if len(s) > smax:
                mid = int(len(s)/2)
                s = s[:mid] + " " + s[mid:]
        ofeatures.append(s)
    return ofeatures

def gen_user_vecs(user_vec, num_items):
    """ given a user vector return:
        user predict maxtrix to match the size of item_vecs """
    user_vecs = np.tile(user_vec, (num_items, 1))
    return user_vecs

def predict_uservec(user_vecs, item_vecs, model, u_s, i_s, scaler):
    """ given a scaled user vector, does the prediction on all movies in scaled print_item_vecs returns
        an array predictions sorted by predicted rating,
        arrays of user and item, sorted by predicted rating sorting index
    """
    y_p = model.predict([user_vecs[:, u_s:], item_vecs[:, i_s:]])
    y_pu = scaler.inverse_transform(y_p)

    if np.any(y_pu < 0):
        print("Error, expected all positive predictions")
    sorted_index = np.argsort(-y_pu, axis=0).reshape(-1).tolist()  #negate to get largest rating first
    sorted_ypu   = y_pu[sorted_index]
    sorted_items = item_vecs[sorted_index]
    sorted_user  = user_vecs[sorted_index]
    return(sorted_index, sorted_ypu, sorted_items, sorted_user)
                
def get_user_vecs(user_id, user_train, item_vecs, user_to_genre):
    """ given a user_id, return:
        user train/predict matrix to match the size of item_vecs
        y vector with ratings for all rated movies and 0 for others of size item_vecs """

    if not user_id in user_to_genre:
        print("error: unknown user id")
        return None
    else:
        user_vec_found = False
        for i in range(len(user_train)):
            if user_train[i, 0] == user_id:
                user_vec = user_train[i]
                user_vec_found = True
                break
        if not user_vec_found:
            print("error in get_user_vecs, did not find uid in user_train")
        num_items = len(item_vecs)
        user_vecs = np.tile(user_vec, (num_items, 1))

        y = np.zeros(num_items)
        for i in range(num_items):  # walk through movies in item_vecs and get the movies, see if user has rated them
            movie_id = item_vecs[i, 0]
            if movie_id in user_to_genre[user_id]['movies']:
                rating = user_to_genre[user_id]['movies'][movie_id]
            else:
                rating = 0
            y[i] = rating
    return(user_vecs, y)


In [250]:
def print_pred_movies(y_p, item, anime_dict, maxcount=10, show_all=False):
    count = 0
    for i in range(len(y_p)):
        if count >= maxcount:
            break
        try:
            anime_id = int(item.iloc[i]['anime_id']) if 'anime_id' in item.columns else i
            
            # Support for both types of anime_dict values
            title = anime_dict.get(anime_id, {}).get('title') if isinstance(anime_dict.get(anime_id), dict) else anime_dict.get(anime_id, "Unknown Title")
            
            rating = float(y_p[i][0])
            genre = item.iloc[i]['genre'] if 'genre' in item.columns else "N/A"

            print(f"{count+1:2d}. {title} (Predicted Rating: {rating:.2f}, Genre: {genre})")
            count += 1

        except Exception as e:
            print(f"[WARN] Skipping row {i} due to error: {e}")

    if show_all:
        print("\nAll Predictions:\n", y_p)


In [255]:
genre_cols = ['Action', 'Adventure','Cars','Comedy','Dementia','Demons','Drama','Ecchi','Fantasy','Game','Harem','Hentai','Historical','Horror','Josei','Kids','Magic','Martial Arts','Mecha','Military','Music','Mystery','Parody','Police','Psychological','Romance','Samurai','School','Sci-Fi','Seinen','Shoujo','Shoujo Ai','Shounen','Shounen Ai','Slice of Life','Space','Sports','Super Power','Supernatural','Thriller','Vampire','Yaoi','Yuri']
user_genre_prefs = {
    "Action": 1,
    "Thriller": 1,
    "Fantasy": 1,
}
user_vec = [user_genre_prefs.get(col, 0) for col in genre_cols]


In [256]:

item_vecs=pd.read_csv('cleaned_converted_data.csv')
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs) 

y_p = model.predict([suser_vecs, sitem_vecs])


y_pu = scalerTarget.inverse_transform(y_p)


sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  
sorted_ypu   = y_pu[sorted_index]
sorted_items = new_sh.iloc[sorted_index]
 

print_pred_movies(sorted_ypu, sorted_items, anime_dict, maxcount=25 )

385/385 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step
 1. Kingyo Chuuihou! (Movie) (Predicted Rating: 6.36, Genre: Action, Comedy, Kids, Shoujo)
 2. Monster High: Kowa-ike Girls (Predicted Rating: 6.36, Genre: Kids, Shoujo, Supernatural)
 3. Watashi to Watashi: Futari no Lotte (Predicted Rating: 6.36, Genre: Kids, Shoujo, Slice of Life)
 4. Kingyo Chuuihou! (Predicted Rating: 6.36, Genre: Comedy, Kids, Shoujo)
 5. Mr. Pen Pen II (Predicted Rating: 6.36, Genre: Comedy, Kids, Shoujo)
 6. Mr. Pen Pen (Predicted Rating: 6.36, Genre: Comedy, Kids, Shoujo)
 7. Ijime: Ikenie no Kyoushitsu (Predicted Rating: 6.36, Genre: Drama, Shoujo)
 8. Glass no Kamen: Sen no Kamen wo Motsu Shoujo (Predicted Rating: 6.36, Genre: Drama, Shoujo)
 9. Glass no Kamen (2005) (Predicted Rating: 6.36, Genre: Drama, Shoujo)
10. Yuki no Taiyou Pilot (Predicted Rating: 6.36, Genre: Drama, Shoujo)
11. Lady Lady!! (1988) (Predicted Rating: 6.36, Genre: Drama, Shoujo)
12. Aru Hi Inu no Kuni kara Tegami ga Kite (Predicted Rating: 6.